In [23]:
from neo4j import GraphDatabase
from datetime import datetime, timedelta

In [ ]:
#docker run -p 7687:7687 -p 7474:7474 --env NEO4J_AUTH=neo4j/neo5jneo5j neo4j

In [15]:
AUTH = ("neo4j", "neo5jneo5j")
URI = "neo4j://localhost"
driver =  GraphDatabase.driver(uri=URI, auth=AUTH)

In [53]:
driver.verify_connectivity()

In [75]:
#items = id, name, price
items = [
    {
        "id": 0,
        "name": "apple",
        "price": 5
    },
    {
        "id": 1,
        "name": "pear",
        "price": 7
    },
    {
        "id": 2,
        "name": "sofa",
        "price": 10
    },
    {
        "id": 3,
        "name": "tv",
        "price": 15
    },
    {
        "id": 4,
        "name": "car",
        "price": 150
    },
    {
        "id": 5,
        "name": "plane",
        "price": 200
    }
]

In [76]:
summary = driver.execute_query(
    """
    UNWIND $rows AS row
    CREATE (:Item {id: row.id, name: row.name, price: row.price})
    """,
    rows=items
).summary

print("Created {nodes_created} nodes in {time} ms.".format(
    nodes_created=summary.counters.nodes_created,
    time=summary.result_available_after
))

Created 6 nodes in 3 ms.


In [77]:
#customers = id, name
customers = [
    {
        "id": 0,
        "name": "Maru"
    },
    {
        "id": 1,
        "name": "Mike"
    },
    {
        "id": 2,
        "name": "Bob"
    },
    {
        "id": 3,
        "name": "Don"
    }
]

In [78]:
summary = driver.execute_query(
    """
    UNWIND $rows AS row
    CREATE (:Customer {id: row.id, name: row.name})
    """,
    rows=customers
).summary

print("Created {nodes_created} nodes in {time} ms.".format(
    nodes_created=summary.counters.nodes_created,
    time=summary.result_available_after
))

Created 4 nodes in 3 ms.


In [79]:
#orders = id, date
orders = [
    {
        "id": 0,
        "date": datetime.today().date() - timedelta(days=1)
    },
    {
        "id": 1,
        "date": datetime.today().date() - timedelta(days=5)
    },
    {
        "id": 2,
        "date": datetime.today().date() - timedelta(days=2)
    },
    {
        "id": 3,
        "date": datetime.today().date() - timedelta(days=1)
    },
    {
        "id": 4,
        "date": datetime.today().date() - timedelta(days=7)
    },
    {
        "id": 5,
        "date": datetime.today().date() - timedelta(days=2)
    },
    {
        "id": 6,
        "date": datetime.today().date() - timedelta(days=2)
    }
]

In [80]:
summary = driver.execute_query(
    """
    UNWIND $rows AS row
    CREATE (:Order {id: row.id, date: row.date})
    """,
    rows=orders
).summary

print("Created {nodes_created} nodes in {time} ms.".format(
    nodes_created=summary.counters.nodes_created,
    time=summary.result_available_after
))

Created 7 nodes in 4 ms.


In [85]:
records, summary, keys = driver.execute_query(
    "MATCH (o:Order) RETURN o",
)

In [86]:
records

[<Record o=<Node element_id='4:e9c8fa48-66f6-4485-a416-f2fb9e8714c5:38' labels=frozenset({'Order'}) properties={'date': neo4j.time.Date(2024, 3, 5), 'id': 0}>>,
 <Record o=<Node element_id='4:e9c8fa48-66f6-4485-a416-f2fb9e8714c5:39' labels=frozenset({'Order'}) properties={'date': neo4j.time.Date(2024, 3, 1), 'id': 1}>>,
 <Record o=<Node element_id='4:e9c8fa48-66f6-4485-a416-f2fb9e8714c5:40' labels=frozenset({'Order'}) properties={'date': neo4j.time.Date(2024, 3, 4), 'id': 2}>>,
 <Record o=<Node element_id='4:e9c8fa48-66f6-4485-a416-f2fb9e8714c5:41' labels=frozenset({'Order'}) properties={'date': neo4j.time.Date(2024, 3, 5), 'id': 3}>>,
 <Record o=<Node element_id='4:e9c8fa48-66f6-4485-a416-f2fb9e8714c5:42' labels=frozenset({'Order'}) properties={'date': neo4j.time.Date(2024, 2, 28), 'id': 4}>>,
 <Record o=<Node element_id='4:e9c8fa48-66f6-4485-a416-f2fb9e8714c5:43' labels=frozenset({'Order'}) properties={'date': neo4j.time.Date(2024, 3, 4), 'id': 5}>>,
 <Record o=<Node element_id='4:e9

In [83]:
customer_order = [
    {
        "customer_id": 0,
        "order_id": 0
    },
    {
        "customer_id": 1,
        "order_id": 2
    },
    {
        "customer_id": 2,
        "order_id": 1
    },
    {
        "customer_id": 3,
        "order_id": 3
    },
    {
        "customer_id": 1,
        "order_id": 4
    },
    {
        "customer_id": 0,
        "order_id": 5
    },
    {
        "customer_id": 1,
        "order_id": 6
    }
]

In [87]:
records, summary, keys = driver.execute_query(
    """
    UNWIND $rows AS row
    MATCH (c:Customer {id: row.customer_id})  
    MATCH (o:Order {id: row.order_id})
    MERGE  (c)-[:BOUGHT]->(o)  
    """, 
    rows=customer_order
)
print(f"Query counters: {summary.counters}.")

Query counters: {}.


In [36]:
order_item = [
    {
        "order_id": 0,
        "item_id": 0
    },
    {
        "order_id": 0,
        "item_id": 3
    },
    {
        "order_id": 0,
        "item_id": 2
    },
    {
        "order_id": 1,
        "item_id": 0
    },
    {
        "order_id": 1,
        "item_id": 5
    },
    {
        "order_id": 2,
        "item_id": 1
    },
    {
        "order_id": 3,
        "item_id": 3
    },{
        "order_id": 3,
        "item_id": 2
    },
    {
        "order_id": 4,
        "item_id": 0
    },
    {
        "order_id": 5,
        "item_id": 1
    },
    {
        "order_id": 5,
        "item_id": 3
    },
    {
        "order_id": 5,
        "item_id": 2
    },
    {
        "order_id": 6,
        "item_id": 5
    },

]

In [88]:
records, summary, keys = driver.execute_query(
    """
    UNWIND $rows AS row
    MATCH (o:Order {id: row.order_id})  
    MATCH (i:Item {id: row.item_id})
    MERGE  (o)-[:CONTAINS]->(i)  
    """, 
    rows=order_item
)
print(f"Query counters: {summary.counters}.")

Query counters: {'_contains_updates': True, 'relationships_created': 13}.


In [89]:
customer_item = [
    {
        "customer_id": 0,
        "item_id": 0
    },
    {
        "customer_id": 0,
        "item_id": 4
    },
    {
        "customer_id": 0,
        "item_id": 5
    },
    {
        "customer_id": 1,
        "item_id": 0
    },
    {
        "customer_id": 1,
        "item_id": 5
    },
    {
        "customer_id": 1,
        "item_id": 2
    },
    {
        "customer_id": 2,
        "item_id": 3
    },
    {
        "customer_id": 2,
        "item_id": 2
    },
    {
        "customer_id": 2,
        "item_id": 0
    },
    {
        "customer_id": 3,
        "item_id": 1
    },
    {
        "customer_id": 3,
        "item_id": 2
    },
    {
        "customer_id": 3,
        "item_id": 5
    }
]

In [90]:
records, summary, keys = driver.execute_query(
    """
    UNWIND $rows AS row
    MATCH (c:Customer {id: row.customer_id})  
    MATCH (i:Item {id: row.item_id})
    MERGE  (c)-[:VIEWED]->(i)  
    """, 
    rows=customer_item
)
print(f"Query counters: {summary.counters}.")

Query counters: {'_contains_updates': True, 'relationships_created': 12}.


In [96]:
records, summary, keys = driver.execute_query(
    """
    MATCH (o:Order {id:$order_id})-[CONTAINS]->(i:Item)
    return i
    """,
    order_id=0
)
records

[<Record i=<Node element_id='4:e9c8fa48-66f6-4485-a416-f2fb9e8714c5:30' labels=frozenset({'Item'}) properties={'price': 10, 'name': 'sofa', 'id': 2}>>,
 <Record i=<Node element_id='4:e9c8fa48-66f6-4485-a416-f2fb9e8714c5:31' labels=frozenset({'Item'}) properties={'price': 15, 'name': 'tv', 'id': 3}>>,
 <Record i=<Node element_id='4:e9c8fa48-66f6-4485-a416-f2fb9e8714c5:28' labels=frozenset({'Item'}) properties={'price': 5, 'name': 'apple', 'id': 0}>>]

In [97]:
records, summary, keys = driver.execute_query(
    """
    MATCH (o:Order {id:$order_id})-[CONTAINS]->(i:Item)
    return sum(i.price)
    """,
    order_id=1
)
records

[<Record sum(i.price)=205>]

In [98]:
records, summary, keys = driver.execute_query(
    """
    MATCH (c:Customer {id:$customer_id})-[BOUGHT]->(o:Order)
    return o
    """,
    customer_id=2
)
records

[<Record o=<Node element_id='4:e9c8fa48-66f6-4485-a416-f2fb9e8714c5:39' labels=frozenset({'Order'}) properties={'date': neo4j.time.Date(2024, 3, 1), 'id': 1}>>]

In [103]:
records, summary, keys = driver.execute_query(
    """
    MATCH (c:Customer {id:$customer_id})-[BOUGHT]->(o:Order)-[CONTAINS]->(i:Item)
    return i
    """,
    customer_id=0
)
records

[<Record i=<Node element_id='4:e9c8fa48-66f6-4485-a416-f2fb9e8714c5:30' labels=frozenset({'Item'}) properties={'price': 10, 'name': 'sofa', 'id': 2}>>,
 <Record i=<Node element_id='4:e9c8fa48-66f6-4485-a416-f2fb9e8714c5:31' labels=frozenset({'Item'}) properties={'price': 15, 'name': 'tv', 'id': 3}>>,
 <Record i=<Node element_id='4:e9c8fa48-66f6-4485-a416-f2fb9e8714c5:29' labels=frozenset({'Item'}) properties={'price': 7, 'name': 'pear', 'id': 1}>>,
 <Record i=<Node element_id='4:e9c8fa48-66f6-4485-a416-f2fb9e8714c5:30' labels=frozenset({'Item'}) properties={'price': 10, 'name': 'sofa', 'id': 2}>>,
 <Record i=<Node element_id='4:e9c8fa48-66f6-4485-a416-f2fb9e8714c5:31' labels=frozenset({'Item'}) properties={'price': 15, 'name': 'tv', 'id': 3}>>,
 <Record i=<Node element_id='4:e9c8fa48-66f6-4485-a416-f2fb9e8714c5:28' labels=frozenset({'Item'}) properties={'price': 5, 'name': 'apple', 'id': 0}>>]

In [104]:
records, summary, keys = driver.execute_query(
    """
    MATCH (c:Customer {id:$customer_id})-[BOUGHT]->(o:Order)-[CONTAINS]->(i:Item)
    return count(i)
    """,
    customer_id=0
)
records

[<Record count(i)=6>]

In [128]:
records, summary, keys = driver.execute_query(
    """
    MATCH (c:Customer {id:$customer_id})-[BOUGHT]->(o:Order)-[CONTAINS]->(i:Item)
    return sum(i.price)
    """,
    customer_id=0
)
records

[<Record sum(i.price)=62>]

In [113]:
records, summary, keys = driver.execute_query(
    """
    MATCH (o:Order)-[CONTAINS]->(i:Item)
    RETURN i.name, count(i) 
    ORDER BY count(i)
    """,
    customer_id=0
)
records

[<Record i.name='plane' count(i)=2>,
 <Record i.name='pear' count(i)=2>,
 <Record i.name='sofa' count(i)=3>,
 <Record i.name='tv' count(i)=3>,
 <Record i.name='apple' count(i)=3>]

In [114]:
records, summary, keys = driver.execute_query(
    """
    MATCH (c:Customer {id:$customer_id})-[VIEWED]->(i:Item)
    return i
    """,
    customer_id=0
)
records

[<Record i=<Node element_id='4:e9c8fa48-66f6-4485-a416-f2fb9e8714c5:33' labels=frozenset({'Item'}) properties={'price': 200, 'name': 'plane', 'id': 5}>>,
 <Record i=<Node element_id='4:e9c8fa48-66f6-4485-a416-f2fb9e8714c5:32' labels=frozenset({'Item'}) properties={'price': 150, 'name': 'car', 'id': 4}>>,
 <Record i=<Node element_id='4:e9c8fa48-66f6-4485-a416-f2fb9e8714c5:28' labels=frozenset({'Item'}) properties={'price': 5, 'name': 'apple', 'id': 0}>>]

In [129]:
records, summary, keys = driver.execute_query(
    """
    MATCH (i:Item {id:$item_id})<-[:CONTAINS]-(o:Order)-[:CONTAINS]->(other_item:Item)
    return other_item
    """,
    item_id=0
)
records

[<Record other_item=<Node element_id='4:e9c8fa48-66f6-4485-a416-f2fb9e8714c5:33' labels=frozenset({'Item'}) properties={'price': 200, 'name': 'plane', 'id': 5}>>,
 <Record other_item=<Node element_id='4:e9c8fa48-66f6-4485-a416-f2fb9e8714c5:30' labels=frozenset({'Item'}) properties={'price': 10, 'name': 'sofa', 'id': 2}>>,
 <Record other_item=<Node element_id='4:e9c8fa48-66f6-4485-a416-f2fb9e8714c5:31' labels=frozenset({'Item'}) properties={'price': 15, 'name': 'tv', 'id': 3}>>]

In [132]:
records, summary, keys = driver.execute_query(
    """
    MATCH (i:Item {id:$item_id})<-[:CONTAINS]-(o:Order)<-[:BOUGHT]-(c:Customer)
    return c
    """,
    item_id=0
)
records

[<Record c=<Node element_id='4:e9c8fa48-66f6-4485-a416-f2fb9e8714c5:35' labels=frozenset({'Customer'}) properties={'name': 'Mike', 'id': 1}>>,
 <Record c=<Node element_id='4:e9c8fa48-66f6-4485-a416-f2fb9e8714c5:36' labels=frozenset({'Customer'}) properties={'name': 'Bob', 'id': 2}>>,
 <Record c=<Node element_id='4:e9c8fa48-66f6-4485-a416-f2fb9e8714c5:34' labels=frozenset({'Customer'}) properties={'name': 'Maru', 'id': 0}>>]

In [150]:
records, summary, keys = driver.execute_query(
    """
    MATCH (c:Customer {id: $customer_id})-[:VIEWED]->(viewed_item:Item)
    WHERE NOT (viewed_item:Item)<-[:CONTAINS]-(:Order)<-[:BOUGHT]-(c)
    return distinct viewed_item
    """,
    customer_id=0
)
records

[<Record viewed_item=<Node element_id='4:e9c8fa48-66f6-4485-a416-f2fb9e8714c5:33' labels=frozenset({'Item'}) properties={'price': 200, 'name': 'plane', 'id': 5}>>,
 <Record viewed_item=<Node element_id='4:e9c8fa48-66f6-4485-a416-f2fb9e8714c5:32' labels=frozenset({'Item'}) properties={'price': 150, 'name': 'car', 'id': 4}>>]